## Preprocessing

In [86]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


What variable(s) are the target(s) for your model?

IS_SUCCESSFUL

What variable(s) are the feature(s) for your model?

APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT

In [87]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [88]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [89]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [90]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_value_counts = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = []
for index, value in application_types_value_counts.items():
    if value < 500:
      application_types_to_replace.append(index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [91]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [92]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df["CLASSIFICATION"].value_counts()
for index, value in classification_value_counts.items():
    if value > 1:
      print(index,value)

C1000 17326
C2000 6074
C1200 4837
C3000 1918
C2100 1883
C7000 777
C1700 287
C4000 194
C5000 116
C1270 114
C2700 104
C2800 95
C7100 75
C1300 58
C1280 50
C1230 36
C1400 34
C7200 32
C2300 32
C1240 30
C8000 20
C7120 18
C1500 16
C1800 15
C6000 15
C1250 14
C8200 11
C1238 10
C1278 10
C1235 9
C1237 9
C7210 7
C2400 6
C1720 6
C4100 6
C1257 5
C1600 5
C1260 3
C2710 3
C0 3
C3200 2
C1234 2
C1246 2
C1267 2
C1256 2


In [93]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_value_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = []
for index, value in classification_value_counts.items():
    if value < 1000:
      classifications_to_replace.append(index)
classifications_to_replace
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [94]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "CLASSIFICATION", "AFFILIATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
# Optimized by adding more training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

## Optimization Attempt 1: Increase number of epochs, increase number of hidden layers, increase number of nodes in layers.

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
# Optimized by adding more neurons. I chose to do 3x the number of inputs
nn.add(tf.keras.layers.Dense(units=129, activation="relu", input_dim=43))

# Optimized by adding more neurons and hidden layers
nn.add(tf.keras.layers.Dense(units=100, activation="relu"))
nn.add(tf.keras.layers.Dense(units=75, activation="relu"))
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 129)               5676      
                                                                 
 dense_1 (Dense)             (None, 100)               13000     
                                                                 
 dense_2 (Dense)             (None, 75)                7575      
                                                                 
 dense_3 (Dense)             (None, 50)                3800      
                                                                 
 dense_4 (Dense)             (None, 25)                1275      
                                                                 
 dense_5 (Dense)             (None, 10)                260       
                                                                 
 dense_6 (Dense)             (None, 5)                 5

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
# Optimized by adding more epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
858/858 [==============================] - 8s 5ms/step - loss: 0.5723 - accuracy: 0.7194
Epoch 2/150
858/858 [==============================] - 4s 5ms/step - loss: 0.5572 - accuracy: 0.7288
Epoch 3/150
858/858 [==============================] - 4s 5ms/step - loss: 0.5554 - accuracy: 0.7290
Epoch 4/150
858/858 [==============================] - 4s 5ms/step - loss: 0.5535 - accuracy: 0.7293
Epoch 5/150
858/858 [==============================] - 4s 5ms/step - loss: 0.5527 - accuracy: 0.7293
Epoch 6/150
858/858 [==============================] - 4s 5ms/step - loss: 0.5510 - accuracy: 0.7320
Epoch 7/150
858/858 [==============================] - 4s 4ms/step - loss: 0.5503 - accuracy: 0.7313
Epoch 8/150
858/858 [==============================] - 4s 5ms/step - loss: 0.5493 - accuracy: 0.7324
Epoch 9/150
858/858 [==============================] - 4s 4ms/step - loss: 0.5488 - accuracy: 0.7322
Epoch 10/150
858/858 [==============================] - 4s 4ms/step - loss: 0.5484 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5975 - accuracy: 0.7290 - 1s/epoch - 5ms/step
Loss: 0.5975016355514526, Accuracy: 0.7290087342262268


## Attempt 2: Binning ASK_AMT

In [95]:
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [96]:
application_df["ASK_AMT"].min()

5000

In [97]:
application_df["ASK_AMT"].max()

8597806340

In [98]:
application_df["ASK_AMT"].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

I'm noticing that most of the dataframe has ASK_AMT = 5000, which is also the minimum value. Perhaps we could just make this column into a binary of those that are 5000 and those that are over.

In [100]:
application_df_2 = application_df.copy()
application_df_2["ASK_AMT_binary"] = 0

for index, row in application_df.iterrows():
    ASK_AMT = application_df_2.loc[index,"ASK_AMT"]
    if ASK_AMT == 5000:
        application_df_2.loc[index,"ASK_AMT_binary"] = "ASK_AMT equals 5000"
    else:
        application_df_2.loc[index,"ASK_AMT_binary"] = "ASK_AMT over 5000"

application_df_2 = pd.get_dummies(application_df_2, columns=["ASK_AMT_binary"])
application_df_2.drop("ASK_AMT", axis=1, inplace=True)
application_df_2

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_binary_ASK_AMT equals 5000,ASK_AMT_binary_ASK_AMT over 5000
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [50]:
# Split our preprocessed data into our features and target arrays
y = application_df_2.IS_SUCCESSFUL.values
X = application_df_2.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [51]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3600      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6061 (23.68 KB)
Trainable params: 6061 (23.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [57]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 743us/step - loss: 0.5696 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 1s 767us/step - loss: 0.5539 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 786us/step - loss: 0.5524 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - 1s 719us/step - loss: 0.5501 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 1s 730us/step - loss: 0.5492 - accuracy: 0.7292
Epoch 6/100
804/804 [==============================] - 1s 719us/step - loss: 0.5481 - accuracy: 0.7313
Epoch 7/100
804/804 [==============================] - 1s 713us/step - loss: 0.5474 - accuracy: 0.7316
Epoch 8/100
804/804 [==============================] - 1s 720us/step - loss: 0.5467 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 1s 727us/step - loss: 0.5463 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 1s 737us/step - l

804/804 [==============================] - 1s 744us/step - loss: 0.5342 - accuracy: 0.7389
Epoch 80/100
804/804 [==============================] - 1s 732us/step - loss: 0.5345 - accuracy: 0.7386
Epoch 81/100
804/804 [==============================] - 1s 723us/step - loss: 0.5340 - accuracy: 0.7400
Epoch 82/100
804/804 [==============================] - 1s 741us/step - loss: 0.5341 - accuracy: 0.7385
Epoch 83/100
804/804 [==============================] - 1s 742us/step - loss: 0.5339 - accuracy: 0.7394
Epoch 84/100
804/804 [==============================] - 1s 757us/step - loss: 0.5341 - accuracy: 0.7391
Epoch 85/100
804/804 [==============================] - 1s 768us/step - loss: 0.5338 - accuracy: 0.7387
Epoch 86/100
804/804 [==============================] - 1s 703us/step - loss: 0.5338 - accuracy: 0.7388
Epoch 87/100
804/804 [==============================] - 1s 718us/step - loss: 0.5339 - accuracy: 0.7388
Epoch 88/100
804/804 [==============================] - 1s 732us/step - loss:

In [58]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7248 - 251ms/epoch - 936us/step
Loss: 0.5606200098991394, Accuracy: 0.724781334400177


## Attempt 3: Removing SPECIAL_CONSIDERATIONS data

In [101]:
application_df["SPECIAL_CONSIDERATIONS_N"].value_counts()

1    34272
0       27
Name: SPECIAL_CONSIDERATIONS_N, dtype: int64

Special considerations that are approved are very rare, so lets try eliminating these as a feature in our model.

In [102]:
application_df_3 = application_df.drop(columns=["SPECIAL_CONSIDERATIONS_N", "SPECIAL_CONSIDERATIONS_Y"])
application_df_3

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [108]:
# Split our preprocessed data into our features and target arrays
y = application_df_3.IS_SUCCESSFUL.values
X = application_df_3.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [109]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [110]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=41))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3360      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5821 (22.74 KB)
Trainable params: 5821 (22.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [112]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 745us/step - loss: 0.5717 - accuracy: 0.7175
Epoch 2/100
804/804 [==============================] - 1s 793us/step - loss: 0.5586 - accuracy: 0.7257
Epoch 3/100
804/804 [==============================] - 1s 720us/step - loss: 0.5554 - accuracy: 0.7274
Epoch 4/100
804/804 [==============================] - 1s 742us/step - loss: 0.5537 - accuracy: 0.7275
Epoch 5/100
804/804 [==============================] - 1s 701us/step - loss: 0.5524 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 1s 710us/step - loss: 0.5520 - accuracy: 0.7284
Epoch 7/100
804/804 [==============================] - 1s 714us/step - loss: 0.5506 - accuracy: 0.7297
Epoch 8/100
804/804 [==============================] - 1s 715us/step - loss: 0.5502 - accuracy: 0.7296
Epoch 9/100
804/804 [==============================] - 1s 708us/step - loss: 0.5494 - accuracy: 0.7304
Epoch 10/100
804/804 [==============================] - 1s 715us/step - l

804/804 [==============================] - 1s 737us/step - loss: 0.5387 - accuracy: 0.7367
Epoch 80/100
804/804 [==============================] - 1s 731us/step - loss: 0.5391 - accuracy: 0.7363
Epoch 81/100
804/804 [==============================] - 1s 747us/step - loss: 0.5389 - accuracy: 0.7363
Epoch 82/100
804/804 [==============================] - 1s 731us/step - loss: 0.5387 - accuracy: 0.7369
Epoch 83/100
804/804 [==============================] - 1s 737us/step - loss: 0.5389 - accuracy: 0.7369
Epoch 84/100
804/804 [==============================] - 1s 740us/step - loss: 0.5391 - accuracy: 0.7366
Epoch 85/100
804/804 [==============================] - 1s 754us/step - loss: 0.5385 - accuracy: 0.7374
Epoch 86/100
804/804 [==============================] - 1s 719us/step - loss: 0.5384 - accuracy: 0.7373
Epoch 87/100
804/804 [==============================] - 1s 727us/step - loss: 0.5385 - accuracy: 0.7371
Epoch 88/100
804/804 [==============================] - 1s 821us/step - loss:

In [113]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5505 - accuracy: 0.7371 - 244ms/epoch - 911us/step
Loss: 0.5505040287971497, Accuracy: 0.7371428608894348


In [114]:
# Save and export your results to an HDF5 file. Name the file AlphabetSoupCharity.h5.
nn.save("Models/AlphabetSoupCharity_Optimization.h5", save_format="h5")

C:\Users\jkaum\anaconda\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
